In [1]:
# imports 

import pandas as pd
import numpy as np
import sys

In [2]:
# constants (drug names)
pain_drugs = ['Codeine',
 'Codeine Phosphate',
 'Codeine Sulfate',  'Fentanyl',
 'Fentanyl ',
 'Fentanyl Citrate',
 'Fentanyl PCA',
 'Fentanyl Patch','FENTANYL',
  'HYDROMORPHONE',
              'Hydrocodone-Acetaminophen',
 'Hydrocodone-Acetaminophen (5mg-500mg',
 'Hydrocodone-Acetaminophen (5mg-500mg)',
 'HYDROmorphone',
 'HYDROmorphone (Dilaudid)',
 'HYDROmorphone Infusion ? Comfort Care Guidelines',
 'HYDROmorphone P.F.',
 'HYDROmorphone PCA',
 'HYDROmorphone PCA (Dilaudid)',
 'HYDROmorphone-HP',
'Hydromorphone',
 'Hydromorphone (Rectal)',
 'Hydromorphone HCl',
 'Hydromorphone P.F.',
 'Hydromorphone PCA',
 'Hydromorphone-HP',
 'Hydromorphone-hp',
'OxycoDONE',
 'OxycoDONE ',
 'OxycoDONE (Immediate Release) ',
 'OxycoDONE Liquid',
 'OxycoDONE-Acetaminophen Elixir',
 'OxycodONE ',
 'OxycodONE (Immediate Release) ',
 'Oxycodone',
 'Oxycodone ',
 'Oxycodone (Sustained Release)',
 'Oxycodone HCl',
 'Oxycodone Liquid',
 'Oxycodone SR (OxyconTIN)',
 'Oxycodone-Acetaminophen',
 'Oxycodone-Acetaminophen (5mg-325mg)',
 'Oxycodone-Acetaminophen Elixir', 'Oxymorphone',
 'Oxymorphone HCl', 'MORPHINE SULFATE', 'Morphine',
 'Morphine ',
 'Morphine (PF)',
 'Morphine Infusion ? Comfort Care Guidelines',
 'Morphine PCA',
 'Morphine S',
 'Morphine SR (MS Contin)',
 'Morphine Sulfa',
 'Morphine Sulfat',
 'Morphine Sulfate',
 'Morphine Sulfate ',
 'Morphine Sulfate (0.5-4mg)',
 'Morphine Sulfate (1-5mg)',
 'Morphine Sulfate (2-4mg)',
 'Morphine Sulfate (2-5mg)',
 'Morphine Sulfate (2-6mg)',
 'Morphine Sulfate (4-8mg)',
 'Morphine Sulfate (Concentrated Oral Soln)',
 'Morphine Sulfate (Immediate Release)',
 'Morphine Sulfate (Oral Soln.)',
 'Morphine Sulfate (PF)',
 'Morphine Sulfate (Syringe)',
 'Morphine Sulfate For Inhalation',
 'Morphine Sulfate IR',
 'Morphine Sulfate PF',
 'Morphine Sulfate SR','Vicodin ',]

In [8]:
# load data
df = pd.read_csv("./data/ADMISSIONS.csv")
df2 = pd.read_csv("./data/PRESCRIPTIONS.csv")

/usr/local/Cellar/ipython/7.0.1/libexec/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
## Result: 
##      p_final and adm_final both represent the same 4026 patients that have been filtered. There are 5 ethnic
##      groups (WHITE, BLACK, ASIAN, HISPANIC, MIDDLE EASTERN) with (3272, 422, 135, 182, 15) patients respectively.
##      p_final: Pandas DataFrame that includes the (opiod) prescription information for 4026 patients
##      adm_final: Pandas DataFrame that includes the admission information for 4026 patients
##

## Need to run the first and second phase of filtration to generate
## generate the Admission table w/ filters and cross checking w/ prescription

# simplify ethnicity names first
df_table = dict()
WHITE = ["WHITE"]
BLACK = ["BLACK"]
ASIAN = ["ASIAN", "PACIFIC"]
HISPANIC = ["HISPANIC", "SOUTH AMERICAN", "PORTUGESE"]
ARAB = ["MIDDLE EASTERN"]
eth_types = [WHITE, BLACK, ASIAN, HISPANIC, ARAB]
for e in eth_types: 
    for i, name in enumerate(e):
        new_df = df.loc[df.ETHNICITY.str.contains(name)]
        df_tmp = new_df if i == 0 else pd.concat([df_tmp, new_df])
    df_tmp.ETHNICITY = e[0]
    df_table[e[0]] = df_tmp
a = list(df_table.values())
df_simple = pd.concat(a)
df_simple = df_simple[df_simple.ADMISSION_TYPE != "NEWBORN"] # Focus on Urgent, Emergency, Elective


# find the common diagnosis between ethnic groups in the dataset
df_diag = df_simple.dropna(axis=0, subset=['DIAGNOSIS'])
df_diag.DIAGNOSIS = df_diag.DIAGNOSIS.apply(lambda x: x.lower())

d_white = set(df_diag[df_diag.ETHNICITY=='WHITE'].DIAGNOSIS.value_counts().index)
d_asian = set(df_diag[df_diag.ETHNICITY=='ASIAN'].DIAGNOSIS.value_counts().index)
d_black = set(df_diag[df_diag.ETHNICITY=='BLACK'].DIAGNOSIS.value_counts().index)
d_hispanic = set(df_diag[df_diag.ETHNICITY=='HISPANIC'].DIAGNOSIS.value_counts().index)
d_mideast = set(df_diag[df_diag.ETHNICITY=='MIDDLE EASTERN'].DIAGNOSIS.value_counts().index)

common = list(d_white.intersection(d_asian, d_black, d_hispanic, d_mideast))

df_common = df_diag[df_diag.DIAGNOSIS.isin(common)]

# generate prescriptions dataframe

p_cohort = df2[df2['DRUG'].isin(pain_drugs)]
HADM = df_common.HADM_ID
p_final = p_cohort[p_cohort.HADM_ID.isin(HADM)]

HADM_drugs = p_final.HADM_ID
adm_final = df_common[df_common.HADM_ID.isin(HADM_drugs)]


# check distribution of diagnosis among ethnic groups
eth = ["WHITE", "BLACK", "ASIAN", "HISPANIC", "MIDDLE EASTERN"]
tmp = []
for e in eth:
    df_tmp = df_common[df_common.ETHNICITY == e]
    tmp.append(df_tmp.DIAGNOSIS.value_counts()/sum(df_tmp.DIAGNOSIS.value_counts().values))
pd.DataFrame(tmp, eth).T

WHITE     BLACK     ASIAN  HISPANIC  \
pneumonia                     0.261024  0.300493  0.276923  0.239669   
chest pain                    0.129626  0.126437  0.107692  0.103306   
intracranial hemorrhage       0.119211  0.057471  0.112821  0.107438   
upper gi bleed                0.090405  0.110016  0.133333  0.140496   
s/p fall                      0.077332  0.027915  0.030769  0.049587   
fever                         0.077332  0.137931  0.128205  0.086777   
hypotension                   0.066696  0.116585  0.025641  0.049587   
pancreatitis                  0.039220  0.031199  0.035897  0.024793   
upper gastrointestinal bleed  0.036339  0.036125  0.025641  0.053719   
blunt trauma                  0.035232  0.014778  0.010256  0.045455   
brain mass                    0.020829  0.006568  0.025641  0.045455   
hemoptysis                    0.017062  0.019704  0.061538  0.004132   
intraparenchymal hemorrhage   0.013517  0.006568  0.015385  0.033058   
pulmonary embolus             0.008199  0.003284  0.005128  0.008264   
atrial fibrillation           0.007977  0.004926  0.005128  0.008264   

                              MIDDLE EASTERN  
pneumonia                           0.190476  
chest pain                          0.095238  
intracranial hemorrhage             0.047619  
upper gi bleed                      0.047619  
s/p fall                            0.047619  
fever                               0.095238  
hypotension                         0.047619  
pancreatitis                        0.047619  
upper gastrointestinal bleed        0.095238  
blunt trauma                        0.047619  
brain mass                          0.047619  
hemoptysis                          0.047619  
intraparenchymal hemorrhage         0.047619  
pulmonary embolus                   0.047619  
atrial fibrillation                 0.047619

In [17]:
p_final

ROW_ID  SUBJECT_ID  HADM_ID  ICUSTAY_ID            STARTDATE  \
614      2981121          11   194540    229441.0  2178-04-17 00:00:00   
618      2981124          11   194540         NaN  2178-04-21 00:00:00   
622      2981129          11   194540         NaN  2178-04-21 00:00:00   
646      2981139          11   194540         NaN  2178-04-24 00:00:00   
668      2981152          11   194540         NaN  2178-05-03 00:00:00   
3247     1550216          68   170467    225771.0  2174-01-01 00:00:00   
3249     1550217          68   170467    225771.0  2174-01-01 00:00:00   
3569     1550244          68   170467         NaN  2173-12-21 00:00:00   
3570     1550245          68   170467         NaN  2173-12-21 00:00:00   
3571     1550246          68   170467         NaN  2173-12-21 00:00:00   
3591     1550251          68   170467         NaN  2173-12-22 00:00:00   
4489     1650203          85   112077    291697.0  2167-07-26 00:00:00   
4525     1650939          85   112077    291697.0  2167-07-26 00:00:00   
4644     1435949          94   140037    213450.0  2176-09-03 00:00:00   
4645     1425887          94   140037    213450.0  2176-09-03 00:00:00   
4661     1425893          94   140037    213450.0  2176-09-04 00:00:00   
4684     1425905          94   140037    213450.0  2176-09-06 00:00:00   
4693     1425915          94   140037    213450.0  2176-09-09 00:00:00   
4703     1425924          94   140037    213450.0  2176-09-10 00:00:00   
4716     1436690          94   140037    213450.0  2176-09-11 00:00:00   
4723     1436691          94   140037    213450.0  2176-09-12 00:00:00   
4725     1436692          94   140037    213450.0  2176-09-12 00:00:00   
4728     1436696          94   140037    213450.0  2176-09-14 00:00:00   
4729     1436697          94   140037    213450.0  2176-09-14 00:00:00   
4798     2238361          96   170324         NaN  2156-03-31 00:00:00   
4817     2237594          96   170324    211943.0  2156-04-01 00:00:00   
4845     2237597          96   170324    211943.0  2156-04-07 00:00:00   
4861     2238319          96   170324    211943.0  2156-04-10 00:00:00   
4866     2238322          96   170324    211943.0  2156-04-10 00:00:00   
4883     2238327          96   170324    211943.0  2156-04-11 00:00:00   
...          ...         ...      ...         ...                  ...   
4154475  4112286       99946   157197         NaN  2161-07-14 00:00:00   
4154676    10547       98831   179520    237542.0  2170-09-12 00:00:00   
4154678    10546       98831   179520    237542.0  2170-09-12 00:00:00   
4154692    10551       98831   179520    237542.0  2170-09-13 00:00:00   
4154694    10550       98831   179520    237542.0  2170-09-13 00:00:00   
4154696    10549       98831   179520    237542.0  2170-09-13 00:00:00   
4154818  3907337       99068   164100    201560.0  2193-05-24 00:00:00   
4155094  4112736       99946   157197    298613.0  2161-07-15 00:00:00   
4155265    10528       98831   179520         NaN  2170-09-07 00:00:00   
4155266    10536       98831   179520         NaN  2170-09-07 00:00:00   
4155272    10534       98831   179520         NaN  2170-09-07 00:00:00   
4155571   149951       98853   189030         NaN  2187-02-09 00:00:00   
4155572   149952       98853   189030         NaN  2187-02-09 00:00:00   
4156152   149800       98853   189030         NaN  2187-02-03 00:00:00   
4156153   149799       98853   189030         NaN  2187-02-03 00:00:00   
4156157   149801       98853   189030         NaN  2187-02-03 00:00:00   
4156161   149941       98853   189030         NaN  2187-02-05 00:00:00   
4156163   149943       98853   189030         NaN  2187-02-05 00:00:00   
4156164   149942       98853   189030         NaN  2187-02-05 00:00:00   
4156414  3488594       98883   144680    235104.0  2102-01-30 00:00:00   
4156428  3488576       98883   144680    235104.0  2102-01-31 00:00:00   
4156429  3488578       98883   144680    235104.0  2102-01-31 00:00:00   
4156430  34885

In [18]:
adm_final

ROW_ID  SUBJECT_ID  HADM_ID            ADMITTIME            DISCHTIME  \
39        473         368   105889  2137-07-11 17:56:00  2137-07-16 16:00:00   
90         88          85   112077  2167-07-25 18:49:00  2167-07-30 15:24:00   
101        99          96   170324  2156-03-31 16:11:00  2156-04-29 15:05:00   
150       189         140   165618  2160-09-21 08:31:00  2160-09-23 12:00:00   
156       195         145   138459  2145-02-15 17:19:00  2145-02-22 13:30:00   
196       152         111   155897  2144-07-01 04:12:00  2144-07-01 14:55:00   
220        10          11   194540  2178-04-16 06:18:00  2178-05-11 19:00:00   
263       411         323   128132  2119-09-21 20:34:00  2119-10-06 18:50:00   
265       413         324   146127  2199-10-16 01:11:00  2199-10-16 17:19:00   
349       619         494   125330  2168-02-20 23:36:00  2168-03-02 16:15:00   
361       631         505   103194  2154-08-23 14:01:00  2154-08-29 11:10:00   
366       636         509   161218  2104-05-06 03:37:00  2104-05-19 16:30:00   
407       516         404   166989  2136-03-22 20:58:00  2136-03-23 00:40:00   
459       240         184   137477  2168-03-13 19:00:00  2168-03-16 15:45:00   
496       277         211   101148  2197-12-17 02:52:00  2197-12-29 14:02:00   
512       293         224   169158  2151-04-01 08:06:00  2151-04-02 10:35:00   
519       300         231   198608  2176-05-14 21:12:00  2176-06-05 15:00:00   
631      1118         901   107952  2195-02-10 01:59:00  2195-02-13 14:20:00   
643      1130         908   134650  2125-11-06 21:00:00  2125-11-08 12:00:00   
677       830         674   130960  2193-06-19 19:40:00  2193-06-27 13:52:00   
769       765         618   185691  2118-04-17 13:15:00  2118-04-29 04:35:00   
833       829         674   116577  2192-05-16 19:05:00  2192-05-23 13:19:00   
838       666         529   112024  2146-05-29 17:18:00  2146-06-02 14:32:00   
858       686         546   107882  2127-04-01 16:33:00  2127-04-07 17:37:00   
910       569         445   137069  2194-11-11 17:03:00  2194-11-14 14:49:00   
933      1037         835   193827  2151-11-24 01:39:00  2151-11-27 17:30:00   
956      1060         854   176032  2140-10-27 04:45:00  2140-10-31 18:45:00   
985      1212         975   114439  2139-02-01 01:02:00  2139-02-26 18:55:00   
1032     1259        1012   148016  2120-05-18 11:09:00  2120-05-24 14:20:00   
1062     1289        1039   149449  2117-04-12 19:56:00  2117-04-24 20:30:00   
...       ...         ...      ...                  ...                  ...   
56421   58958       99937   129380  2128-05-11 15:16:00  2128-05-20 13:35:00   
56634   56415       91989   186557  2161-07-31 21:28:00  2161-08-07 13:23:00   
56961   57647       95893   126615  2184-02-03 09:20:00  2184-02-11 11:28:00   
57112   58525       98597   197435  2189-10-21 11:37:00  2189-10-22 14:10:00   
57390   58009       96908   138027  2145-02-02 23:13:00  2145-02-04 17:07:00   
57391   58010       96908   100994  2145-09-04 11:10:00  2145-09-07 14:17:00   
57850   56498       92281   163710  2118-09-19 14:03:00  2118-10-08 15:20:00   
58211   56660       92855   162192  2121-03-17 03:05:00  2121-03-20 12:54:00   
58321   55925       90541   124428  2156-09-05 04:53:00  2156-09-12 00:39:00   
58436   58869       99647   192055  2147-11-24 14:17:00  2147-12-01 16:45:00   
58530   57132       94260   166585  2161-07-28 23:13:00  2161-07-30 07:50:00   
58542   57899       96645   199054  2104-03-01 12:57:00  2104-03-05 13:28:00   
58574   57931       96699   186382  2132-06-08 15:37:00  2132-06-14 16:20:00   
58675   57151       94321   117033  2112-02-27 01:02:00  2112-03-01 15:30:00   
53107   52580       79964   152161  2170-05-25 20:55:00  2170-05-29 18:08:00   
20471   21522       17617   128901  2194-11-14 21:57:00  2194-12-18 18:00:00   
41950   43048       50366   127513  2195-08-10 00:39:00  2195-08-28 14:20:00   
43658   45509       57889   111845  2102-04-06 17:06:00  2102-04-14 15:15:00   
43792  